In [3]:
#!uv pip install jupyterlab==4.2.1 openai==1.38.0 langchain-ollama==0.1.1 langchain-openai==0.1.19 litellm==1.42.11 icecream==2.1.3 tiktoken==0.7.0

In [ ]:
#!uv pip freeze > requirements.txt 

In [4]:
# Warning control
import warnings
#warnings.simplefilter("ignore", category=Warning)
warnings.filterwarnings("ignore")

In [5]:
from IPython.display import Image, Markdown,display
import json
import os
import shutil
import psutil
import re
import requests
from getpass import getpass
import openai
from os import environ

IN_NOTEBOOK = any(["jupyter-lab" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

if environ.get('OPENAI_API_KEY') is None:
    print('Environment variable not found. Setting values...')
    os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v1']['credential'] 
    os.environ["TOGETHERAI_API_KEY"] = CREDS['together-ai']['key']['credential']
    os.environ['ANTHROPIC_API_KEY'] = CREDS['anthropic']['key']['credential']
    os.environ["SERPER_API_KEY"] = CREDS['serper_dev']['key']['credential']
    #openai.api_key = CREDS['OpenAI']['v2']['credential'] 

Secrets (JSON string):  ········


Environment variable not found. Setting values...


In [3]:
import openai

# List available models

see https://platform.openai.com/docs/models

**or via bash:**

%%bash

curl https://api.openai.com/v1/models -H "Authorization: Bearer $OPENAI_API_KEY"

In [6]:
# Get list of available models
client = openai.OpenAI()
model_list = client.models.list()
for model in model_list:
  print(model.id)

dall-e-3
gpt-4-1106-preview
dall-e-2
tts-1-hd-1106
tts-1-hd
gpt-4o-mini-2024-07-18
gpt-4-0125-preview
babbage-002
gpt-4-turbo-preview
text-embedding-3-small
text-embedding-3-large
tts-1
gpt-3.5-turbo
whisper-1
gpt-4o-2024-05-13
text-embedding-ada-002
gpt-3.5-turbo-16k
davinci-002
gpt-4-turbo-2024-04-09
tts-1-1106
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-3.5-turbo-1106
gpt-4o-mini
gpt-4o
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct
gpt-4-0613
gpt-4


# Simple LLM calls: via OpenAI API

In [8]:
# OpenAI

client = openai.OpenAI(
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "user",
      "content": "Hello, how are you?",
    }
  ],
  model="gpt-4o-mini",
  temperature=0.7,
  max_tokens=1024
)

print(chat_completion.choices[0].message.content)

Hello! I'm just a program, but I'm here and ready to help you. How can I assist you today?


In [7]:
# Mixtral - via together.ai

client = openai.OpenAI(
  api_key=os.environ.get("TOGETHERAI_API_KEY"),
  base_url='https://api.together.xyz',
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "user",
      "content": "Hello, how are you?",
    }
  ],
  model="mistralai/Mixtral-8x7B-Instruct-v0.1",
  temperature=0.7,
  max_tokens=1024
)

print(chat_completion.choices[0].message.content)

 Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you with any language-related questions you have. Is there something specific you would like to know or practice?


In [10]:
# Ollama

client = openai.OpenAI(
    api_key="not-needed",
    base_url = "http://localhost:11434/v1"
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "user",
      "content": "Hello, how are you?",
    }
  ],
  model="llama3.1:8b", # this field is currently unused
  temperature=0.7,
  max_tokens=1024
)

print(chat_completion.choices[0].message.content)

I'm doing well, thank you for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How about you, how's your day going?


In [ ]:
# LM Studio

client = openai.OpenAI(
    api_key="not-needed",
    base_url="http://localhost:1234/v1"
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "user",
      "content": "Hello, how are you?",
    }
  ],
  model="local-model", # this field is currently unused
  temperature=0.7,
  max_tokens=1024
)

print(chat_completion.choices[0].message.content)

# Simple LLM calls: via Langchain

llm is defined by factory 'get_llm' - can easilly switched between Ollama and OpenAI

In [15]:
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

def get_llm(llm_type):
    if llm_type == "ollama":
        return ChatOllama(model="llama3.1", temperature=0)
    else:
        return ChatOpenAI(temperature=0, model="gpt-4o-mini")

def get_embeddings(embedding_type):
    if embedding_type == "ollama":
        return OllamaEmbeddings(model="llama3.1:8b")
    else:
        return OpenAIEmbeddings()

In [16]:
llm = get_llm('openai')
result = llm.invoke(
    "Question: What do dogs like to eat?"
    "Answer: Let's think step by step."
)
print(result.content)
print(result)

Sure! When considering what dogs like to eat, we can break it down into several categories:

1. **Commercial Dog Food**: Most dog owners feed their pets commercial dog food, which comes in dry (kibble) and wet (canned) forms. These foods are formulated to meet a dog's nutritional needs.

2. **Meat**: Dogs are carnivores by nature, so they typically enjoy eating meat. Common options include:
   - Chicken
   - Beef
   - Turkey
   - Fish

3. **Fruits and Vegetables**: Many dogs enjoy certain fruits and vegetables, which can be healthy treats. Some popular options include:
   - Carrots
   - Apples (without seeds)
   - Blueberries
   - Sweet potatoes
   - Green beans

4. **Grains**: While dogs don't require grains in their diet, some may enjoy them. Common grains include:
   - Rice
   - Oats
   - Barley

5. **Treats**: Dogs often love treats, which can be store-bought or homemade. Popular treats include:
   - Biscuits
   - Chews (like rawhide or dental chews)
   - Peanut butter (make sure i

In [17]:
llm = get_llm('ollama')
result = llm.invoke(
    "Question: What do dogs like to eat?"
    "Answer: Let's think step by step."
)
print(result.content)
print(result)

Let's break down what dogs like to eat, step by step.

**Step 1: Protein is a Must**
Dogs are carnivores, which means they primarily thrive on protein-rich foods. Their natural diet consists of meat, bones, and organs from animals. So, we can conclude that dogs love to eat protein-rich foods.

**Step 2: Meat and Poultry**
Within the category of protein-rich foods, dogs tend to enjoy meat and poultry products like chicken, beef, lamb, and pork. These are common ingredients in dog food and treats.

**Step 3: Variety is Key**
While dogs have a strong preference for meat-based foods, they also appreciate variety in their diet. This can include fruits, vegetables, and grains, but these should not be the primary components of their meals.

**Step 4: Avoid Human Foods (Mostly)**
While it's tempting to share our snacks with our furry friends, many human foods are toxic or unhealthy for dogs. For example, chocolate, onions, garlic, grapes, and raisins can be poisonous to dogs. It's best to stic

# Simple LLM calls: via lite-llm

In [12]:
from litellm import completion

In [13]:
# OpenAI
response = completion(
  model="gpt-3.5-turbo",
  #model="together_ai/togethercomputer/Llama-2-7B-32K-Instruct",
  #model="together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response.choices[0].message.content)
print(response)

Hello! I'm just a computer program, so I don't have feelings in the same way humans do, but I'm here and ready to help you. How can I assist you today?
ModelResponse(id='chatcmpl-9s957SpRTw3jnPfHCJX9XAsBhHm9U', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! I'm just a computer program, so I don't have feelings in the same way humans do, but I'm here and ready to help you. How can I assist you today?", role='assistant', tool_calls=None, function_call=None))], created=1722691649, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=Usage(completion_tokens=39, prompt_tokens=13, total_tokens=52), service_tier=None)


In [14]:
# Ollama
response = completion(
  # api_base="http://localhost:11434",   # seems to work even without this parameter!
  model="ollama/llama3.1:8b", 
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response.choices[0].message.content)
print(response)

I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How about you? How's your day going?
ModelResponse(id='chatcmpl-b7a2835a-ed2b-49b5-b1f0-54fbdc67e75c', choices=[Choices(finish_reason='stop', index=0, message=Message(content="I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How about you? How's your day going?", role='assistant', tool_calls=None, function_call=None))], created=1722691731, model='ollama/llama3.1:8b', object='chat.completion', system_fingerprint=None, usage=Usage(prompt_tokens=19, completion_tokens=57, total_tokens=76))


In [ ]:
# LM Studio
# see https://litellm.vercel.app/docs/providers/openai_compatible
response = completion(
  api_base="http://localhost:1234/v1",   # seems to work even without this parameter!
  model="openai/just-a-dummy-model", # prefix 'openai/' is required! model name is currently unused in LM Studio
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response.choices[0].message.content)
print(response)

# Function Calling

## via Langchain

In [19]:
from typing import List

def validate_user(user_id: int, addresses: List) -> bool:
    """Validate user using historical addresses. 

        Args:
        user_id: (int) the user ID.
        addresses: Previous addresses.
    """
    return True

llm = get_llm('ollama') # see functions above
llm_with_tools = llm.bind_tools([validate_user])

result = llm_with_tools. invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)
print(result.tool_calls)
#print(result.content)
#print(result)

[{'name': 'validate_user', 'args': {'addresses': '[{"street":"Fake St","city":"Boston","state":"MA"},{"street":"Pretend Boulevard","city":"Houston","state":"TX"}]', 'user_id': 123}, 'id': 'a49c8db3-7be0-4fdb-b58e-890adab99bb0', 'type': 'tool_call'}]


## via litellm

In [ ]:
# Function Calling

# see https://litellm.vercel.app/docs/completion/function_call

# via Huggingface?
# https://litellm.vercel.app/docs/providers/huggingface
# https://huggingface.co/Trelis/Mixtral-8x7B-Instruct-v0.1-function-calling-v3
# https://huggingface.co/Trelis/Mistral-7B-Instruct-v0.1-function-calling-v2

# via Anyscale?
# https://docs.litellm.ai/docs/providers/anyscale
# https://www.anyscale.com/blog/anyscale-endpoints-json-mode-and-function-calling-features

import os, litellm
from litellm import completion

# IMPORTANT - Set this to TRUE to add the function to the prompt for Non OpenAI LLMs
litellm.add_function_to_prompt = True

# The real function is not needed for the LLM. It may be called after the LLM call (not in this code!)
def get_current_weather(location):
  if location == "Boston, MA":
    return "The weather is 12F"

functions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]

messages = [
    {"role": "user", "content": "What is the weather like in Boston?"}
]

llm_model = "gpt-3.5-turbo"
#llm_model = "gpt-3.5-turbo-1106"
#llm_model = "gpt-4"
#llm_model = "claude-2" # ??? claude answers in response.choices[0]['message']['content']
#llm_model = "together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1"


response = completion(model=llm_model, messages=messages, functions=functions)

function_found = hasattr(response.choices[0]['message'], 'function_call')
if function_found == True:
  function_call = response.choices[0]['message']['function_call']
  function_call_name = function_call.name
  function_call_arguments = function_call.arguments
  print(function_call_arguments)
else:
  if llm_model == "claude-2":
    print(response.choices[0]['message']['content'])
  else:
    print('No function found')

print('\n')
print(response.choices[0]['message']['content'])
print('\n')
print(json.dumps(response.json(), indent=2))

In [ ]:
import os, litellm
from litellm import completion

litellm.add_function_to_prompt = False

functions_string = """[
  {
    "name": "get_risk_data",
    "description": "Daten zum Versicherungsnehmer und seiner Risiken",
    "parameters": {
      "type": "object",
      "properties": {
        "Versicherungssumme": {
          "type": "integer",
          "description": "Versicherungssumme oder Deckungssumme der Versicherung, z. B. 100000"
        },
        "Versicherungsnehmer_Name": {
          "type": "string",
          "description": "Name des versicherten Unternehmens, z. B. 'Testfirma GmbH'"
        },
        "Versicherungsnehmer_Anschrift": {
          "type": "string",
          "description": "Anschrift des versicherten Unternehmens: Strasse, PLZ, Ort, z. B. 'Musterstr. 10, 10000 Berlin'"
        },
        "Versicherungsnehmer_Branche": {
          "type": "string",
          "description": "Branche bzw. Tätigkeitsbereich des versicherten Unternehmens, z. B. 'Textilproduktion'"
        },
        "Versicherungsnehmer_Umsatz": {
          "type": "string",
          "description": "Jahresumsatz des versicherten Unternehmens, z. B. 23000000"
        },
        "Vorschäden_benannt": {
          "type": "string",
          "description": "Informationen zu früheren Schadensfällen, z. B. 'Ja, in 2019 gab es einen Schaden mit folgenden Details'"
        },
      },
      "required": ["Versicherungssumme"]
    }
  }
]
"""

functions=eval(functions_string)

email_test_text = """wir hoffen, Sie hatten einen angenehmen Feiertag.

Zu unserer besteMeiermen Kundenverbindung Zorrsen Beteiligungsgesellschaft mbH bitten wir um Ihre Quotierung zur D&O-Versicherung per 01.01.2024.

Die aktuelle Versicherungssumme beträgt 5,5 Mio. € zu einer Jahresprämie in Höhe von 7.920 € netto.
Der Umsatz im letzten Jahr betrug 78,5 Millionen EUR.

Beigefügt erhalten Sie die bisherigen Vertragsunterlagen sowie ein Organigramm. Versichert werden soll die Zorrsen Beteiligungsgesellschaft mbH als VN und die darunterliegenden Tochtergesellschaften (gelb markiert).

Es gibt einen laufenden D&O-Schadensfall aufgrund einer Inanspruchnahme eines ehemaligen Geschäftsführers, welcher im Jahr 2017 entlassen wurde. Ihm wird vorgeworfen, für ein damaliges Bauvorhaben das Budget deutlich überzogen zu haben.

Für Ihre Rückmeldung vielen Dank im Voraus.
"""

messages = [
    {"role": "user", "content": "Extrahiere die Risikodaten: " + email_test_text}
]

llm_model = "gpt-3.5-turbo"
#llm_model = "gpt-3.5-turbo-1106"
#llm_model = "gpt-4"
#llm_model = "claude-2" # ??? claude answers in response.choices[0]['message']['content']
#llm_model = "together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1"


response = completion(model=llm_model, messages=messages, functions=functions)

function_found = hasattr(response.choices[0]['message'], 'function_call')
if function_found == True:
  function_call = response.choices[0]['message']['function_call']
  function_call_name = function_call.name
  function_call_arguments = function_call.arguments
  print(function_call_arguments)
else:
  if llm_model == "claude-2":
    print(response.choices[0]['message']['content'])
  else:
    print('No function found')

print('\n')
print(response.choices[0]['message']['content'])
print('\n')
print(json.dumps(response.json(), indent=2))

In [ ]:
print(response.choices[0]['message']['content'])

# Text splitting

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

content = "This is a piece of text. With some text and some more text."

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 10,
    chunk_overlap  = 0,
    length_function = len,
)

chunks = text_splitter.split_text(content)

print('Number of text chunks: ', len(chunks))

for chunk in chunks:
    print(chunk)

# Token counting

- https://platform.openai.com/tokenizer
- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

In [ ]:
import tiktoken

content = "This is a piece of text. With some text and some more text."

enc = tiktoken.get_encoding("cl100k_base")
token_count = len(enc.encode(content))

print('Number of tokens: ', token_count)